Tarefa: criar uma funcao que recebe conj1 e conj2 (de preferencia via loc 
em um mesmo df) e sinaliza de alguma forma potenciais duplicatas de conj1 em
conj2.

In [1]:
import pandas as pd
import os
import numpy
import pickle

from ivpy import attach,show,montage,histogram,scatter,compose
from ivpy.extract import extract
from ivpy.reduce import pca,tsne,umap
from ivpy.cluster import cluster
from ivpy.extract import norm
from sklearn.metrics import adjusted_rand_score as adjrand
from annoy import AnnoyIndex
from ivpy.data import _typecheck,_pathfilter,_featfilter,seq_types
from ivpy.plot import show

2021-11-23 13:22:53.034737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 13:22:53.034784: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
metadata = pd.read_csv("/mnt/e/Tasks/similarity/Files/metadata.csv")

#PREPARE DF

In [3]:
DIR = "/mnt/e/Tasks/Vistas Situadas - Imagens/images_cropped"
df = pd.DataFrame()
df["id"] = [id.split(".")[0] for id in os.listdir(DIR) if id.endswith(".jpg")]
df["filename"] = [os.path.join(DIR, filename) for filename in os.listdir(DIR) if filename.endswith(".jpg")]

In [4]:
df

,id,filename
0,001AAN005121_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
1,001AAN005122_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
2,001AAN005123_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
3,001AAN005124_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
4,001AAN005125_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
...,...,...
5599,P015 BR RJIHGB 125IG 0008,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
5600,P015 BR RJIHGB 125IG 0009,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
5601,P015 BR RJIHGB 125IG 0010,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...
5602,P015 BR RJIHGB 125IG 0011,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...


In [5]:
#Add information regarding the status of if the img 

df['geolocated'] = metadata["Source ID"].apply(lambda x: int(1) if x.endswith("_geo.jpg") else 0 )

df.set_index("id",inplace=True)
metadata.set_index("Source ID",inplace=True)
df.reset_index(inplace=True)
df

,id,filename,geolocated
0,001AAN005121_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
1,001AAN005122_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
2,001AAN005123_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
3,001AAN005124_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
4,001AAN005125_geo,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
...,...,...,...
5599,P015 BR RJIHGB 125IG 0008,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
5600,P015 BR RJIHGB 125IG 0009,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
5601,P015 BR RJIHGB 125IG 0010,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0
5602,P015 BR RJIHGB 125IG 0011,/mnt/e/Tasks/Vistas Situadas - Imagens/images_...,0


In [6]:
attach(df, "filename")

In [7]:
X = extract('neural')
feature_matrix=norm(X)

2021-11-23 13:22:58.119759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-23 13:22:58.119794: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-23 13:22:58.119809: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-F74BU77): /proc/driver/nvidia/version does not exist
2021-11-23 13:22:58.119953: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


102981632/102967424 [==============================] - 8s 0us/step
5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 

#save pickels

In [8]:
df.to_pickle("/mnt/e/Tasks/similarity/files/situated_views_df_gray.pickle")
feature_matrix.to_pickle("/mnt/e/Tasks/similarity/files/situated_views_features_gray.pickle")